# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [13]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [14]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))


/home/jovyan/work/Projects/project-2


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [15]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
    # read with pandas
    file_df = pd.read_csv(f, index_col=None, header=0)
    full_data_rows_list.append(file_df)
    
full_data = pd.concat(full_data_rows_list, axis=0, ignore_index=True)

# uncomment the code below if you would like to get total number of rows 
print(len(full_data), "rows in source")

# uncomment the code below if you would like to check to see what the list of event data rows will look like
# display(full_data)

full_data = full_data[full_data.artist.notnull()]
print(len(full_data), "rows in destination")

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
full_data.to_csv(
    'event_datafile_new.csv', 
    index=False, 
    columns=['artist','firstName','gender','itemInSession','lastName','length','level','location','sessionId','song','userId'], 
    encoding = 'utf8'
)
display(full_data)

8056 rows in source
6820 rows in destination


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540340e+12,139,You Gotta Be,200,1.541110e+12,8.0
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540340e+12,139,Flat 55,200,1.541110e+12,8.0
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540340e+12,139,Quem Quiser Encontrar O Amor,200,1.541110e+12,8.0
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540340e+12,139,Eriatarka,200,1.541110e+12,8.0
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540340e+12,139,Becoming Insane,200,1.541110e+12,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8050,Foo Fighters,Logged In,Rylan,M,57,George,271.38567,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,The Pretender,200,1.543600e+12,16.0
8051,Timbiriche,Logged In,Rylan,M,58,George,202.60526,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,Besos De Ceniza,200,1.543600e+12,16.0
8052,A Perfect Circle,Logged In,Rylan,M,59,George,206.05342,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,Rose,200,1.543600e+12,16.0
8053,Anberlin,Logged In,Rylan,M,60,George,348.68200,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,The Haunting,200,1.543600e+12,16.0


In [16]:
# check the number of rows in your csv file
print(sum(1 for line in open('event_datafile_new.csv', 'r')))


6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="./images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [17]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['cassandra'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [18]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
""")

except Exception as e:
    print(e)

#### Set Keyspace

In [19]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Create tables

In [20]:
# SELECT artist, song, length FROM listened_songs_events WHERE session_id = 338 and item_in_session = 4
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS listened_songs_events (
            session_id bigint,
            item_in_session int,
            user_id bigint,
            first_name text,
            last_name text,
            gender text,
            location text,
            level text,
            song text,
            length float,
            artist text,
            PRIMARY KEY ((session_id, item_in_session))
        )
    """)
except Exception as e:
    print(e)


# SELECT artist, song, first_name, last_name FROM listened_songs_by_user_by_session WHERE session_id = 182 and user_id = 10
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS listened_songs_by_user_by_session (
            session_id bigint,
            user_id bigint,
            item_in_session int,
            first_name text,
            last_name text,
            song text,
            artist text,
            PRIMARY KEY ((session_id, user_id), item_in_session)
        )
    """)
except Exception as e:
    print(e)

    
# SELECT first_name, last_name FROM users_by_songs_listened WHERE song = 'All Hands Against His Own'
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS users_by_songs_listened (
            song text,
            user_id bigint,
            first_name text,
            last_name text,
            PRIMARY KEY (song, user_id)
        )
    """)
except Exception as e:
    print(e)
                    

## Reading data from file

In [21]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'
dtypes = {
    'sessionId': 'int64',
    'itemInSession': 'int32',
    'userId': 'int64',
    'firstName': 'str',
    'lastName': 'str',
    'gender': 'str',
    'location': 'str',
    'level': 'str',
    'song': 'str',
    'length': 'float',
    'artist': 'str',
}
csvcontent = pd.read_csv('event_datafile_new.csv', dtype=dtypes, encoding = 'utf8')

## Inserting data into the tables

In [22]:
print("Inserting data... Wait a bit")
for idx, row in csvcontent.iterrows():
    try:
        insert_listened_songs_events = """
            INSERT INTO listened_songs_events (
                session_id,
                item_in_session,
                user_id,
                first_name,
                last_name,
                gender,
                location,
                level,
                song,
                length,
                artist
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        
        session.execute(insert_listened_songs_events, (
            row.sessionId,
            row.itemInSession,
            row.userId,
            row.firstName,
            row.lastName,
            row.gender,
            row.location,
            row.level,
            row.song,
            row.length,
            row.artist
        ))
    except Exception as e:
        print(e)
    
    try:
        insert_listened_songs_events = """
            INSERT INTO listened_songs_by_user_by_session (
                session_id,
                user_id,
                item_in_session,
                first_name,
                last_name,
                song,
                artist
            ) VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        
        session.execute(insert_listened_songs_events, (
            row.sessionId,
            row.userId,
            row.itemInSession,
            row.firstName,
            row.lastName,
            row.song,
            row.artist
        ))
    except Exception as e:
        print(e)
    
    try:
        insert_listened_songs_events = """
            INSERT INTO users_by_songs_listened (
                song,
                user_id,
                first_name,
                last_name
            ) VALUES (%s, %s, %s, %s)
        """
        
        session.execute(insert_listened_songs_events, (
            row.song,
            row.userId,
            row.firstName,
            row.lastName
        ))
    except Exception as e:
        print(e)

print("Data inserted")

Inserting data... Wait a bit
Data inserted


## Select statements to verify that the data have been inserted into each table

#### Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [23]:
query_1 = "SELECT artist, song, length FROM listened_songs_events WHERE session_id = 338 and item_in_session = 4"
print(query_1)
try:
    query_1_result = session.execute(query_1)
except Exception as e:
    print(e)
    
display(pd.DataFrame(list(query_1_result)))

SELECT artist, song, length FROM listened_songs_events WHERE session_id = 338 and item_in_session = 4


,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


#### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [24]:
query_2 = "SELECT artist, song, first_name, last_name FROM listened_songs_by_user_by_session WHERE session_id = 182 and user_id = 10"
print(query_2)
try:
    query_2_result = session.execute(query_2)
except Exception as e:
    print(e)
    
display(pd.DataFrame(list(query_2_result)))

SELECT artist, song, first_name, last_name FROM listened_songs_by_user_by_session WHERE session_id = 182 and user_id = 10


,artist,song,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


#### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [25]:
query_3 = "SELECT first_name, last_name FROM users_by_songs_listened WHERE song = 'All Hands Against His Own'"
print(query_3)
try:
    query_3_result = session.execute(query_3)
except Exception as e:
    print(e)
    
display(pd.DataFrame(list(query_3_result)))

SELECT first_name, last_name FROM users_by_songs_listened WHERE song = 'All Hands Against His Own'


,first_name,last_name
0,Jacqueline,Lynch
1,Sara,Johnson
2,Tegan,Levine


## Drop the tables before closing out the sessions

In [26]:
## TO-DO: Drop the table before closing out the sessions
try:
    session.execute("DROP TABLE listened_songs_events")
    session.execute("DROP TABLE listened_songs_by_user_by_session")
    session.execute("DROP TABLE users_by_songs_listened")
except Exception as e:
    print(e)

## Close the session and cluster connection¶

In [27]:
session.shutdown()
cluster.shutdown()